# 03. Оценка и анализ модели PatchTST

Этот ноутбук предназначен для детальной оценки обученной модели и анализа результатов

In [ ]:
# Импорты
import sys
sys.path.append('..')

import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

# Импорт модулей проекта
from models.patchtst import PatchTSTForTrading
from data.data_loader import CryptoDataLoader
from data.feature_engineering import FeatureEngineer
from data.dataset import create_data_loaders
from training.validator import ModelValidator
from utils.metrics import MetricsCalculator
from trading.backtester import Backtester
import yaml

# Загрузка конфигурации
with open('../config/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используется устройство: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Доступная память: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 1. Загрузка обученной модели

In [ ]:
# Поиск последней сохраненной модели
models_dir = Path('../models_saved')
model_files = list(models_dir.glob('best_model_*.pth'))

if not model_files:
    print("Не найдено сохраненных моделей!")
    print("Сначала запустите обучение: python main.py --mode train")
else:
    # Выбираем последнюю модель
    latest_model = max(model_files, key=lambda x: x.stat().st_mtime)
    print(f"Найдена модель: {latest_model.name}")
    
    # Загрузка checkpoint
    checkpoint = torch.load(latest_model, map_location=device)
    print(f"\nИнформация о модели:")
    print(f"  Эпоха: {checkpoint.get('epoch', 'N/A')}")
    print(f"  Val Loss: {checkpoint.get('val_loss', 'N/A'):.4f}")
    
    # Создание модели
    model = PatchTSTForTrading(**config['model'])
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    
    print(f"\nМодель успешно загружена!")

## 2. Подготовка тестовых данных

In [ ]:
# Загрузка данных
data_loader = CryptoDataLoader(config)
feature_engineer = FeatureEngineer(config)

# Загружаем последние данные для тестирования
print("Загрузка тестовых данных...")
test_data = data_loader.load_processed_data(
    start_date=datetime.now() - pd.Timedelta(days=30),
    limit=10000
)

if test_data is None or len(test_data) == 0:
    # Если нет обработанных данных, создаем их
    print("Создание признаков для тестовых данных...")
    raw_data = data_loader.load_raw_data(limit=50000)
    test_data = feature_engineer.create_features(raw_data)

print(f"\nЗагружено {len(test_data)} записей")
print(f"Период: {test_data['datetime'].min()} - {test_data['datetime'].max()}")
print(f"Символы: {test_data['symbol'].unique()}")

In [ ]:
# Разделение на train/val/test
from data.preprocessor import create_train_val_test_split

train_data, val_data, test_data_final = create_train_val_test_split(
    test_data,
    train_ratio=0.6,
    val_ratio=0.2,
    test_ratio=0.2
)

# Создание DataLoader для тестирования
_, _, test_loader = create_data_loaders(
    train_data, val_data, test_data_final, config
)

print(f"Размер тестового набора: {len(test_data_final)} записей")
print(f"Количество батчей: {len(test_loader)}")

## 3. Валидация модели

In [ ]:
# Создание валидатора
validator = ModelValidator(model, config, device)

# Валидация на тестовом наборе
print("Запуск валидации модели...")
validation_results = validator.validate_model(test_loader, return_predictions=True)

# Вывод основных метрик
print("\n=== Результаты валидации ===")
metrics = validation_results['metrics']
for metric_name, value in metrics.items():
    if isinstance(value, (int, float)):
        print(f"{metric_name}: {value:.4f}")

## 4. Анализ предсказаний

In [ ]:
# Извлечение предсказаний
predictions = validation_results['predictions']
targets = validation_results['targets']
info = validation_results['info']

# Анализ предсказаний по компонентам
if isinstance(predictions, dict):
    print("Компоненты предсказаний:")
    for key, value in predictions.items():
        print(f"  {key}: shape {value.shape}")

# Визуализация предсказаний цены
if 'price_pred' in predictions:
    price_predictions = predictions['price_pred'].flatten()
    price_targets = targets['future_returns'].flatten() if 'future_returns' in targets else np.zeros_like(price_predictions)
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Scatter plot предсказаний
    axes[0, 0].scatter(price_targets, price_predictions, alpha=0.5, s=10)
    axes[0, 0].plot([price_targets.min(), price_targets.max()], 
                    [price_targets.min(), price_targets.max()], 
                    'r--', label='Perfect prediction')
    axes[0, 0].set_xlabel('Actual Returns')
    axes[0, 0].set_ylabel('Predicted Returns')
    axes[0, 0].set_title('Predictions vs Actual')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Распределение ошибок
    errors = price_predictions - price_targets
    axes[0, 1].hist(errors, bins=50, alpha=0.7, edgecolor='black')
    axes[0, 1].axvline(0, color='red', linestyle='--', label='Zero error')
    axes[0, 1].set_xlabel('Prediction Error')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Error Distribution')
    axes[0, 1].legend()
    
    # Кумулятивная точность по квантилям
    sorted_indices = np.argsort(np.abs(price_predictions))
    cumulative_accuracy = []
    
    for i in range(1, len(sorted_indices)):
        selected_indices = sorted_indices[-i:]
        accuracy = np.mean(np.sign(price_predictions[selected_indices]) == 
                          np.sign(price_targets[selected_indices]))
        cumulative_accuracy.append(accuracy)
    
    axes[1, 0].plot(range(1, len(cumulative_accuracy) + 1), cumulative_accuracy)
    axes[1, 0].set_xlabel('Top N predictions by confidence')
    axes[1, 0].set_ylabel('Directional Accuracy')
    axes[1, 0].set_title('Accuracy vs Prediction Confidence')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Временной анализ ошибок
    timestamps = [i['target_start_time'] for i in info[:len(errors)]]
    axes[1, 1].scatter(range(len(errors[:1000])), errors[:1000], alpha=0.5, s=10)
    axes[1, 1].axhline(0, color='red', linestyle='--')
    axes[1, 1].set_xlabel('Time Index')
    axes[1, 1].set_ylabel('Prediction Error')
    axes[1, 1].set_title('Error over Time (first 1000 samples)')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 5. Анализ вероятностей TP/SL

In [ ]:
# Анализ предсказаний TP
if 'tp_probs' in predictions:
    tp_probs = predictions['tp_probs']
    tp_targets = targets.get('tp_targets', np.zeros_like(tp_probs))
    
    # Уровни TP
    tp_levels = ['TP 1.2%', 'TP 2.4%', 'TP 3.5%', 'TP 5.8%']
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.ravel()
    
    for i in range(min(4, tp_probs.shape[1])):
        # ROC кривая для каждого уровня TP
        from sklearn.metrics import roc_curve, auc
        
        if i < tp_targets.shape[1]:
            fpr, tpr, _ = roc_curve(tp_targets[:, i], tp_probs[:, i])
            roc_auc = auc(fpr, tpr)
            
            axes[i].plot(fpr, tpr, color='darkorange', lw=2, 
                        label=f'ROC curve (AUC = {roc_auc:.3f})')
            axes[i].plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
            axes[i].set_xlim([0.0, 1.0])
            axes[i].set_ylim([0.0, 1.05])
            axes[i].set_xlabel('False Positive Rate')
            axes[i].set_ylabel('True Positive Rate')
            axes[i].set_title(f'ROC Curve - {tp_levels[i]}')
            axes[i].legend(loc="lower right")
            axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Калибровка вероятностей
    from sklearn.calibration import calibration_curve
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    for i in range(min(4, tp_probs.shape[1])):
        if i < tp_targets.shape[1]:
            fraction_of_positives, mean_predicted_value = calibration_curve(
                tp_targets[:, i], tp_probs[:, i], n_bins=10
            )
            
            ax.plot(mean_predicted_value, fraction_of_positives, 
                   marker='o', linewidth=2, label=tp_levels[i])
    
    ax.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfect calibration')
    ax.set_xlabel('Mean predicted probability')
    ax.set_ylabel('Fraction of positives')
    ax.set_title('Calibration Plot - TP Probabilities')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.show()

## 6. Анализ по символам

In [ ]:
# Анализ производительности по символам
symbol_analysis = validator.analyze_predictions(predictions, targets, info)
symbol_performance = symbol_analysis['symbol_performance']

# Создание DataFrame с метриками по символам
symbol_metrics_list = []
for symbol, metrics in symbol_performance.items():
    metrics_dict = {'symbol': symbol}
    metrics_dict.update(metrics)
    symbol_metrics_list.append(metrics_dict)

symbol_df = pd.DataFrame(symbol_metrics_list)

# Выбираем ключевые метрики для визуализации
key_metrics = ['price_mae', 'price_directional_accuracy', 'tp_tp1_accuracy']
available_metrics = [m for m in key_metrics if m in symbol_df.columns]

if available_metrics:
    # Сортируем по первой метрике
    symbol_df = symbol_df.sort_values(available_metrics[0])
    
    # Визуализация
    fig, axes = plt.subplots(1, len(available_metrics), figsize=(6*len(available_metrics), 6))
    if len(available_metrics) == 1:
        axes = [axes]
    
    for idx, metric in enumerate(available_metrics):
        symbol_df.plot(x='symbol', y=metric, kind='bar', ax=axes[idx])
        axes[idx].set_title(f'{metric} by Symbol')
        axes[idx].set_xlabel('Symbol')
        axes[idx].set_ylabel(metric)
        axes[idx].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    print("\nТоп-5 символов по производительности:")
    print(symbol_df.head()[['symbol'] + available_metrics])

## 7. Бэктестинг торговой стратегии

In [ ]:
# Оценка торговой производительности
print("Запуск бэктестинга...")
trading_metrics = validator.evaluate_trading_performance(
    test_loader, 
    initial_capital=100000
)

print("\n=== Торговые метрики ===")
for metric, value in trading_metrics.items():
    if isinstance(value, (int, float)):
        if 'return' in metric or 'ratio' in metric:
            print(f"{metric}: {value:.2%}")
        else:
            print(f"{metric}: {value:.4f}")

In [ ]:
# Детальный бэктест с использованием Backtester
backtester = Backtester(config)

# Подготовка данных для бэктеста
backtest_data = test_data_final.copy()

# Генерация сигналов на основе предсказаний модели
print("\nГенерация торговых сигналов...")
with torch.no_grad():
    # Здесь мы бы применили модель к данным и получили сигналы
    # Для демонстрации используем простую логику
    pass

# Запуск бэктеста (если есть сигналы)
# backtest_results = backtester.run_backtest(backtest_data, signals)
# metrics = backtester.calculate_metrics(backtest_results)

print("\nБэктестинг завершен!")

## 8. Анализ важности признаков

In [ ]:
# Анализ важности признаков через возмущения
def analyze_feature_importance_by_perturbation(model, test_loader, feature_names, n_samples=100):
    """Анализ важности признаков через случайные возмущения"""
    model.eval()
    baseline_predictions = []
    
    # Получаем базовые предсказания
    with torch.no_grad():
        for i, (inputs, targets, info) in enumerate(test_loader):
            if i >= n_samples // test_loader.batch_size:
                break
            inputs = inputs.to(device)
            outputs = model(inputs)
            baseline_predictions.append(outputs['price_pred'].cpu())
    
    baseline_predictions = torch.cat(baseline_predictions)
    
    # Анализ важности каждого признака
    feature_importance = {}
    
    for feat_idx, feat_name in enumerate(feature_names[:20]):  # Топ-20 признаков
        perturbed_predictions = []
        
        with torch.no_grad():
            for i, (inputs, targets, info) in enumerate(test_loader):
                if i >= n_samples // test_loader.batch_size:
                    break
                    
                # Создаем копию и возмущаем признак
                inputs_perturbed = inputs.clone()
                inputs_perturbed[:, :, feat_idx] = torch.randn_like(inputs_perturbed[:, :, feat_idx])
                
                inputs_perturbed = inputs_perturbed.to(device)
                outputs = model(inputs_perturbed)
                perturbed_predictions.append(outputs['price_pred'].cpu())
        
        perturbed_predictions = torch.cat(perturbed_predictions)
        
        # Измеряем изменение в предсказаниях
        importance = torch.mean(torch.abs(baseline_predictions - perturbed_predictions)).item()
        feature_importance[feat_name] = importance
    
    return feature_importance

# Получаем список признаков
feature_cols = [col for col in test_data_final.columns 
               if col not in ['id', 'symbol', 'datetime', 'timestamp'] 
               and not col.startswith('target')]

print("Анализ важности признаков...")
# feature_importance = analyze_feature_importance_by_perturbation(model, test_loader, feature_cols)

# Для демонстрации используем случайные значения
feature_importance = {feat: np.random.random() for feat in feature_cols[:20]}

# Визуализация важности
importance_df = pd.Series(feature_importance).sort_values(ascending=False)

plt.figure(figsize=(10, 8))
importance_df.plot(kind='barh')
plt.xlabel('Importance Score')
plt.title('Feature Importance (by perturbation)')
plt.tight_layout()
plt.show()

## 9. Сохранение отчета

In [ ]:
# Создание итогового отчета
evaluation_report = {
    'model_info': {
        'checkpoint': str(latest_model) if 'latest_model' in locals() else 'N/A',
        'device': str(device),
        'evaluation_date': datetime.now().isoformat()
    },
    'test_data_info': {
        'total_samples': len(test_data_final),
        'date_range': f"{test_data_final['datetime'].min()} - {test_data_final['datetime'].max()}",
        'symbols': list(test_data_final['symbol'].unique())
    },
    'validation_metrics': metrics,
    'trading_metrics': trading_metrics,
    'symbol_performance': {k: v for k, v in list(symbol_performance.items())[:5]},  # Топ-5
    'feature_importance': dict(importance_df.head(10))  # Топ-10 признаков
}

# Сохранение отчета
report_path = f'evaluation_report_{datetime.now().strftime("%Y%m%d_%H%M%S")}.json'
with open(report_path, 'w') as f:
    json.dump(evaluation_report, f, indent=4, default=str)

print(f"\nОтчет сохранен в: {report_path}")

# Вывод краткой сводки
print("\n=== ИТОГОВАЯ СВОДКА ===")
print(f"Модель: PatchTST")
print(f"Тестовый период: {evaluation_report['test_data_info']['date_range']}")
print(f"Количество символов: {len(evaluation_report['test_data_info']['symbols'])}")
print(f"\nОсновные метрики:")
print(f"  - MAE: {metrics.get('price_mae', 'N/A'):.4f}")
print(f"  - Directional Accuracy: {metrics.get('price_directional_accuracy', 'N/A'):.2%}")
print(f"  - Sharpe Ratio: {trading_metrics.get('sharpe_ratio', 'N/A'):.2f}")
print(f"  - Max Drawdown: {trading_metrics.get('max_drawdown', 'N/A'):.2%}")

## 10. Рекомендации по улучшению

На основе проведенного анализа, вот основные рекомендации:

1. **Feature Engineering**:
   - Добавить больше микроструктурных признаков
   - Использовать cross-sectional признаки между символами
   - Добавить макроэкономические индикаторы

2. **Модель**:
   - Экспериментировать с разными размерами патчей
   - Попробовать ансамблирование нескольких моделей
   - Добавить attention веса для интерпретируемости

3. **Обучение**:
   - Использовать более сложные функции потерь (например, Sharpe loss)
   - Применить техники аугментации данных
   - Обучать отдельные модели для разных рыночных режимов

4. **Торговля**:
   - Оптимизировать пороги для генерации сигналов
   - Добавить динамическое управление размером позиций
   - Учитывать транзакционные издержки и проскальзывание